In [2]:
from beexai.dataset.dataset import Dataset
from beexai.explaining import CaptumExplainer
from beexai.metrics.get_results import get_all_metrics

ModuleNotFoundError: No module named 'beexai'

In [ ]:
# Basic imports
import os
import sys
import time
import timeit
import random
import pickle
import re
from itertools import product
import warnings

# System path modification
sys.path.insert(0, '..')

# Data handling
import pandas as pd
import numpy as np

# Machine learning imports
from sklearn.compose import ColumnTransformer 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer, MissingIndicator


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

from sklearn.inspection import permutation_importance

# Custom modules
from src.train import *
from src.functions import *
from src.plots import *
from src.dataset import *
from src.multixgboost import *
from src.wrapper import *

# Visualizatiokn 
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning and machine learning specific 
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
import xgboost as xgb
import shap

# Ignore warnings
warnings.filterwarnings("ignore")

# Print CUDA availability for PyTorch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


# Load TabNet

In [ ]:
retrain = False

data = load_pickle_data_palettes()

results_pickle_folder = "../pickle/"

# Unpack data
df_X, df_y, df_all, df_FinalCombination = data["df_X"], data["df_y"], data["df_all"], data["df_FinalCombination"]
df_select_features = data["df_select_features"]
df_select_features.columns = ["RNA", "CSF", "DNA", "MRIth"]

# Unpack feature selections
select_RNA, select_CSF, select_gene, select_MRIthickness = df_select_features.T.values

# Unpack colormaps
full_palette, gender_palette, dx_palette = data["colormaps"].values()

In [ ]:
idx_train = list(df_X.isna().any(axis=1))
idx_test = list(~df_X.isna().any(axis=1))

set_intersect_rid = set(df_all[idx_train].RID).intersection(set(df_all[idx_test].RID))
intersect_rid_idx = df_all.RID.isin(set_intersect_rid)

for i, bool_test in enumerate(idx_test): 
    if intersect_rid_idx.iloc[i] & bool_test:
        idx_test[i] = False
        idx_train[i] = True

df_X_train = df_X.loc[idx_train]
df_X_test = df_X.loc[idx_test]

df_y_train = df_y.loc[idx_train]
df_y_test = df_y.loc[idx_test]

c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

In [ ]:
with open("../pickle/training_2_dict_results.pickle", "rb") as input_file:
    all_dict_results = pickle.load(input_file)

In [ ]:
if retrain : 
    tabnet_model =  TabNetModelWrapper(n_a=8, n_d=8)#   XGBoostRegressor() #  TabNetModelWrapper(n_a=8, n_d=8)
else :  
    tabnet_model = all_dict_results[9]["model"] # Corresponds to TabNet trained in notebook #3  

continuous_model  = imputer = KNNImputer(n_neighbors=5)
name_continuous_imputer  = "KNNImputer_5"
ordinal_model =  KNNImputer(n_neighbors=1)
name_ordinal_imputer = "KNNImputer"
name_model = "TabNetRegressor" # "XGBoostRegressor"#  "TabNetRegressor"

## Choose an explanation 

In [1]:

# 1. Prepare your data for BEExAI
df = df_X_test.copy()
df['target'] = df_y_test
data = Dataset(df, target_col='target')
X_train, X_test, y_train, y_test = data.get_train_test(test_size=0.0)  # use your manual split if preferred

# 2. Train or load a regression model
from beexai.training.train import Trainer
trainer = Trainer('GradientBoosting', task='regression', model_params={'input_dim': X_train.shape[1]})
trainer.train(X_train, y_train)

# 3. Initialize the explainer
explainer = CaptumExplainer(trainer.model, task='regression', method='IntegratedGradients', sklearn=False)
explainer.init_explainer()

# 4. Compute attributions (on your test data)
attributions = explainer.compute_attributions(X_test, 'mydata', 'GBT', 'IntegratedGradients', './output/explain/', use_abs=False)

# 5. Evaluate metrics
get_all_metrics(X_test, trainer.model.predict(X_test.values))

NameError: name 'df_X_test' is not defined